<center><img src="https://emojipedia-us.s3.dualstack.us-west-1.amazonaws.com/thumbs/240/apple/285/woman-artist_1f469-200d-1f3a8.png" width="120">
</center>

### <center>Use this notebook to run your DALL-E server</center>
### <center> [DALL-E Playground Repository](https://github.com/saharmor/dalle-playground) </center>

#####<center>This notebook is based on the amazing [DALL·E Mini](https://github.com/borisdayma/dalle-mini) by [Boris Dayma](https://github.com/borisdayma) et al.</center>

## Install dependencies

In [13]:
# only if we need to reset:
# !rm -rf dalle-playground
!git clone https://github.com/dcsan/dalle-playground.git
!pip install -r dalle-playground/backend/requirements.txt
# !npm install -g localtunnel


fatal: destination path 'dalle-playground' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-yr7jw3ym
  Running command git clone -q https://github.com/huggingface/transformers.git /tmp/pip-req-build-yr7jw3ym
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Cloning https://github.com/patil-suraj/vqgan-jax.git to /tmp/pip-req-build-pk6gpipn
  Running command git clone -q https://github.com/patil-suraj/vqgan-jax.git /tmp/pip-req-build-pk6gpipn
  Cloning https://github.com/borisdayma/dalle-mini.git to /tmp/pip-req-build-hq7m9o1m
  Running command git clone -q https://github.com/borisdayma/dalle-mini.git /tmp/pip-req-build-hq7m9o1m
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel me

## Choose a model

* **DALL-E Mini**
  * The original DALL-E Mini. Fastest yet suboptimal results
  * Runs well on a 4GB GPU and the Google Colab free tier
* **DALL-E Mega**
  * The advanced version of DALL-E Mini. Requires more compute and VRAM
  * Runs well on a 8GB GPU and recommended on the Google Colab Pro tier
* **DALL-E Mega Full**
  * The most performant DALL-E version. Requires significantly more computre and VRAM
  * Runs well on a 12GB GPU and recommended on the Google Colab Pro+ tier


In [14]:
# # get/install ngrok
# !wget --no-check-certificate -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
# !wget --no-check-certificate -nc https://eyalgruss.com/fomm/ngrok-stable-linux-amd64.zip
# !unzip -n ngrok-stable-linux-amd64.zip
# !pkill -f ngrok
# !nohup /content/ngrok http -region us -inspect=false $port &
# !npm install -g localtunnel

# !npm install -g npm

# check we have a GPU
!nvidia-smi



changed 14 packages, and audited 202 packages in 2s

11 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities
Fri Jun 24 13:09:19 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-----------------------

In [15]:
#@title Select a model - remember to execute cell after selecting!

dalle_model = 'Mega_full' #@param ["Mini", "Mega", "Mega_full"]

# Run the backend web server

In [18]:
!pip install pyngrok

from threading import Thread
from pyngrok import ngrok
import time
import sys
import requests
app_port = 8000
update_url = 'http://revel.alfalabs.xyz/api/revel/imagenServer/set'

def restart_ngrok():
  # close existing bg processes
  tunnels = ngrok.get_tunnels()
  for tunnel in tunnels:
    print('closing tunnel', tunnel.public_url)
    ngrok.disconnect(tunnel.public_url)
  # free token
  ngrok.set_auth_token("2B22jRiuHD49bVL6PCGoTdyswXV_3WGVzsg68cnDQWmBUgsSe")
  ngcon = ngrok.connect(app_port)
  puburl = ngcon.public_url
  print('ngrok public_url\n', puburl)
  ngrok_process = ngrok.get_ngrok_process()
  start = time.time()
  json = {'url': puburl}
  result = requests.post(update_url, json = json)
  print(result.status_code, result.json())

  try:
    # Block until CTRL-C or some other terminating event
    # time.sleep(10)
    ngrok_process.proc.wait()
  except KeyboardInterrupt:
    print(" Shutting down server")
    ngrok.kill()
  finally:
    now = time.time()
    uptime = round(now - start)
    print(f'uptime {uptime}s')
    print(f'uptime {round(100 * uptime/60) / 100} mins')

# restart_ngrok() # test

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:

def app():
  print(f"Selected DALL-E Model - [{dalle_model}]")
  !python dalle-playground/backend/app.py --port {app_port} --model_version {dalle_model} --save_to_disk true --img_format jpeg --output_dir generations
  print(f'dalle starting on {app_port}')

if __name__=='__main__':
  print('start thread')
  t1 = Thread(target = app)
  a = t1.start()
  # !lt --port {app_port}
  restart_ngrok()  
else:
  print('not main')


start thread
Selected DALL-E Model - [Mega_full]
--> Starting DALL-E Server. This might take up to two minutes.
ngrok public_url
 http://cff9-34-90-122-10.ngrok.io
200 {'ok': True, 'msg': 'updating AiGen server to: `[http://cff9-34-90-122-10.ngrok.io]`'}
tcmalloc: large alloc 10350092288 bytes == 0x7f84a0e78000 @  0x7f88c0c431e7 0x4a3940 0x5b438c 0x5ea94f 0x5939cb 0x594cd3 0x5d0ecb 0x59aeca 0x515655 0x549e0e 0x4bca8a 0x59c019 0x595ef6 0x5134a6 0x549e0e 0x593fce 0x548ae9 0x5127f1 0x4bc98a 0x532b86 0x594a96 0x515600 0x549576 0x604173 0x5f5506 0x5f8c6c 0x5f9206 0x64faf2 0x64fc4e 0x7f88c0840c87 0x5b621a
--> DALL-E Server is up and running!
--> Model selected - DALL-E ModelSize.MEGA_FULL
 * Serving Flask app 'app' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
INFO:werkzeug: * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8000
 * Running on http://172.28.0.2:8000 (Press CTRL+C to quit)
INFO:werkzeug:127.0.0.1 - - [24

## Now use the updated URL
for the main application

it should look like this:

```
ngrok public_url
 http://xxxx-12-34-567-89.ngrok.io
```

## 🎉 Let the fun begin ✨
